# IMPORTS

In [5]:
import pandas as pd, numpy as np
import sklearn
import scipy.sparse
import warnings
warnings.filterwarnings('ignore')

from ipynb.fs.full.Data_Preprocessing import remove_stop_words, make_feature_vec

## UNCOMMENT TO PROCESS DATA FROM SCRATCH

In [6]:
# data = pd.read_csv('../data/train.csv')
# labels=(data["toxic"] | data["severe_toxic"] | data["obscene"] | data["threat"] | data["insult"] | data["identity_hate"])
# train_data = data[0:100000]
# test_data = data[100001:]
# train_labels = labels[0:100000]
# test_labels = labels[100001:]

### CLEANING DATA 
# clean_comments = remove_stop_words(data)
# ret = make_feature_vec(clean_comments)

# vocab = ret['vocab']
# train_feature_vectors_sparse = ret['train_feature_vectors_sparse'][0:100000]
# test_feature_vectors_sparse = ret['train_feature_vectors_sparse'][100001:]



# USE ALREADY PROCESSED DATA

In [17]:
labels = pd.read_csv('../data/Processed/labels.csv')["# labels"]
train_labels = labels[0:100000]
test_labels = labels[100001:]

train_feature_vectors_sparse_load = scipy.sparse.load_npz('../data/Processed/sparse_train_matrix.npz')

train_feature_vectors_sparse = train_feature_vectors_sparse_load[0:100000]
test_feature_vectors_sparse = train_feature_vectors_sparse_load[100001:]
vocab = pd.read_csv('../data/Processed/vocab.csv')["# vocab"]

In [9]:
print("Shape of train vec:", train_feature_vectors_sparse.shape)
print("Shape of test vec:", test_feature_vectors_sparse.shape)

Shape of train vec: (100000, 168595)
Shape of test vec: (59570, 168595)


# TRAINING NB CLASSIFIER

In [10]:
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB()
clf.fit(train_feature_vectors_sparse, train_labels)

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

###### TEST DATA PROCESSING

In [11]:
##### STRAY BLOCK FOR LATER USAGE #####

# # creating sparse representation of test feature vectors
# vocab_mapping=set(zip([i for i in range (0,len(vocab))],vocab))
# print("Starting to create bag of words...")
# vectorizer_test = CountVectorizer(analyzer="word", tokenizer=None, preprocessor=None, vocabulary=vocab_mapping)
# test_feature_vectors_sparse = vectorizer_test.fit_transform(clean_test_comments)
# test_feature_vectors_full = test_feature_vectors_sparse.toarray()

# print("Bag of words created.")

# PREDICTION

In [12]:
predictions = clf.predict(test_feature_vectors_sparse)

print(sum(test_labels)/len(test_labels))
print(sum(train_labels)/len(train_labels))

0.101292596945
0.10191


# PERFORMANCE METRICS

In [13]:
confmat = sklearn.metrics.confusion_matrix(test_labels, predictions)

In [14]:
precision = confmat[1,1]/(confmat[1,1] + confmat[1,0])
recall = confmat[1,1]/(confmat[1,1] + confmat[0,1])
fScore = (2*precision*recall)/(recall+precision)
ccr = (confmat[0,0] + confmat[1,1])/(sum(sum(confmat)))

print("Precision = ",precision)
print("Recall = ",recall)
print("F_score = ",fScore)
print("CCR = ",ccr)

Precision =  0.712462711303
Recall =  0.588904109589
F_score =  0.644817759112
CCR =  0.92049689441


In [15]:
confmat

array([[50535,  3001],
       [ 1735,  4299]])